In [188]:
import numpy as np
import pandas as pd
import fuzzymatcher
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [189]:
df = pd.read_csv('title3.csv', encoding = "ISO-8859-1")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64398 entries, 0 to 64397
Data columns (total 24 columns):
id                 64398 non-null int64
title              64398 non-null object
imdb_index         29 non-null object
kind_id            64398 non-null int64
production_year    64398 non-null int64
imdb_id            0 non-null float64
phonetic_code      55232 non-null object
episode_of_id      59636 non-null float64
season_nr          54786 non-null float64
episode_nr         54757 non-null float64
series_years       3896 non-null object
md5sum             64398 non-null object
id.1               64398 non-null int64
movie_id           64398 non-null int64
company_id         64398 non-null int64
company_type_id    64398 non-null int64
note               32211 non-null object
id.2               64398 non-null int64
name               64398 non-null object
country_code       64398 non-null object
imdb_id.1          0 non-null float64
name_pcode_nf      64398 non-null object
name

In [190]:
df_right = df[df.kind_id != 7.0]

df_right.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4762 entries, 0 to 64397
Data columns (total 24 columns):
id                 4762 non-null int64
title              4762 non-null object
imdb_index         29 non-null object
kind_id            4762 non-null int64
production_year    4762 non-null int64
imdb_id            0 non-null float64
phonetic_code      4762 non-null object
episode_of_id      0 non-null float64
season_nr          0 non-null float64
episode_nr         0 non-null float64
series_years       3896 non-null object
md5sum             4762 non-null object
id.1               4762 non-null int64
movie_id           4762 non-null int64
company_id         4762 non-null int64
company_type_id    4762 non-null int64
note               3046 non-null object
id.2               4762 non-null int64
name               4762 non-null object
country_code       4762 non-null object
imdb_id.1          0 non-null float64
name_pcode_nf      4762 non-null object
name_pcode_sf      4762 non-null

In [191]:
df_right.describe()

,id,kind_id,production_year,imdb_id,episode_of_id,season_nr,episode_nr,id.1,movie_id,company_id,company_type_id,id.2,imdb_id.1
count,4.762000e+03,4762.000000,4762.000000,0.0,0.0,0.0,0.0,4.762000e+03,4.762000e+03,4762.000000,4762.000000,4762.000000,0.0
mean,2.248175e+06,2.181856,1954.571819,NaN,NaN,NaN,NaN,1.490394e+06,2.248175e+06,17376.355523,1.345233,17376.355523,NaN
std,1.160822e+06,0.385767,2.962623,NaN,NaN,NaN,NaN,1.339174e+06,1.160822e+06,42789.638159,0.475494,42789.638159,NaN
min,8.305000e+03,2.000000,1950.000000,NaN,NaN,NaN,NaN,2.335000e+03,8.305000e+03,66.000000,1.000000,66.000000,NaN
25%,1.330491e+06,2.000000,1952.000000,NaN,NaN,NaN,NaN,4.525392e+05,1.330491e+06,83.000000,1.000000,83.000000,NaN
50%,2.520334e+06,2.000000,1955.000000,NaN,NaN,NaN,NaN,6.968295e+05,2.520334e+06,822.000000,1.000000,822.000000,NaN
75%,2.908518e+06,2.000000,1957.000000,NaN,NaN,NaN,NaN,2.705399e+06,2.908518e+06,6753.000000,2.000000,6753.000000,NaN
max,4.584399e+06,3.000000,1959.000000,NaN,NaN,NaN,NaN,4.857393e+06,4.584399e+06,340615.000000,2.000000,340615.000000,NaN


In [192]:
df_left = pd.read_csv('enc.csv', encoding = "ISO-8859-1")
df_left.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1732 entries, 0 to 1731
Data columns (total 12 columns):
Number                 1732 non-null int64
Program Title          1732 non-null object
Program Type           1731 non-null object
Program Genre          1731 non-null object
Network                1704 non-null object
First Air Month        131 non-null float64
First Air Day          123 non-null float64
First Air Year         1732 non-null int64
Last Air Month         2 non-null float64
Last Air Day           2 non-null float64
Last Air Year          895 non-null object
Program Description    1732 non-null object
dtypes: float64(4), int64(2), object(6)
memory usage: 162.5+ KB


In [193]:
df_left["Program Type"].value_counts()

Series               1549
Pilot                 120
Unaired Pilot          31
Series                 10
Live Test Program       5
Experimental            3
Test Film               3
S eries                 2
Pilot                   2
Serial                  2
{Smes                   1
S enes                  1
Test Program            1
 {Series                1
Name: Program Type, dtype: int64

In [194]:
df_right = df_right.astype(str).apply(lambda x: x.str.lower())
df_left = df_left.astype(str).apply(lambda x: x.str.lower())

In [195]:
df_right.drop(df_right.columns[[2,5,7,8,9, 20]], axis=1, inplace=True)

In [196]:
df_right.head()

,id,title,kind_id,production_year,phonetic_code,series_years,md5sum,id.1,movie_id,company_id,company_type_id,note,id.2,name,country_code,name_pcode_nf,name_pcode_sf,md5sum.1
0,63168,abe burrows' almanac,2,1950,a1624,1950-????,73511f5cea12328331c9f3cfdca47334,15574,63168,66,1,nan,66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
1,73174,action in the afternoon,2,1953,a2353,1953-????,7f69e95080d2d8e28d20f13d0ecf1f65,17407,73174,66,1,(1953) (usa) (tv) (original airing),66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
18,78738,adventure showcase,2,1959,a3153,1959-????,cedfa222a39b180f357804d025fc0a8c,18489,78738,66,1,(1959) (usa) (tv),66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
20,80134,adventures of wild bill hickok,2,1951,a3153,1951-1958,fde05a8ca4e780263489033614ae4c4c,18905,80134,66,1,(1951) (usa) (tv) (original airing),66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab
60,91087,air power,2,1956,a616,1956-1958,11aa9136a6caf0118a531270533aa991,21301,91087,66,1,(1956-1957) (usa) (tv),66,columbia broadcasting system (cbs),[us],c4516,c4516,7ae3ee174d2d4b06fc3d2ba6d7dc2fab


In [197]:
df_right.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4762 entries, 0 to 64397
Data columns (total 18 columns):
id                 4762 non-null object
title              4762 non-null object
kind_id            4762 non-null object
production_year    4762 non-null object
phonetic_code      4762 non-null object
series_years       4762 non-null object
md5sum             4762 non-null object
id.1               4762 non-null object
movie_id           4762 non-null object
company_id         4762 non-null object
company_type_id    4762 non-null object
note               4762 non-null object
id.2               4762 non-null object
name               4762 non-null object
country_code       4762 non-null object
name_pcode_nf      4762 non-null object
name_pcode_sf      4762 non-null object
md5sum.1           4762 non-null object
dtypes: object(18)
memory usage: 706.9+ KB


In [198]:
df_left.drop(df_left.head().columns[[5,6,8,9]], axis=1, inplace=True)

In [199]:
df_left.head()

,Number,Program Title,Program Type,Program Genre,Network,First Air Year,Last Air Year,Program Description
0,5137,mary kay and johnny.,series,comedy,"dumont, nbc, cbs",1947,1950,the marital misadventures of mary kay and joh...
1,7531,the small fry club.,series,children,dumont,1947,1951,"bob emery hosts a program of stories, game co..."
2,1738,crawford mystery theater.,series,game,dumont,1947,1952,the alternate title for public prosecutor (wh...
3,6636,public prosecutor.,series,mystery,"syn., dumont",1947,1952,the syndicated version of the series (1947-48...
4,4323,juvenile jury.,series,children,"nbc, cbs",1947,1955,the program features a panel of five children...


In [200]:
df_right["title"].head()

0               abe burrows' almanac
1            action in the afternoon
18                adventure showcase
20    adventures of wild bill hickok
60                         air power
Name: title, dtype: object

In [201]:
df_left["Program Title"] = df_left["Program Title"].str.replace('[^\w\s]','')

In [202]:
df_left["Program Title"].head()

0          mary kay and johnny 
1           the small fry club 
2     crawford mystery theater 
3            public prosecutor 
4                juvenile jury 
Name: Program Title, dtype: object

In [203]:
(process.extract(x, df_left, scorer= fuzz.ratio) for x in df_right)

<generator object <genexpr> at 0x1248f87d8>

In [204]:
#This works but needs tweaking

names_array=[]
ratio_array=[]

def match_names(wrong_names,correct_names):
    for row in wrong_names:
        x=process.extractOne(row, correct_names)
        names_array.append(x[0])
        ratio_array.append(x[1])
    return names_array,ratio_array
 
 
#imdb names dataset

wrong_names=df_right["title"].dropna().values
 
#enc names dataset
correct_names=df_left["Program Title"].dropna().values
 
name_match,ratio_match=match_names(wrong_names,correct_names)
 
df_right['correct_name']=pd.Series(name_match)
df_right['name_ratio']=pd.Series(ratio_match)
 
 
print(df_right[['title','correct_name','name_ratio']].head(10))

                              title               correct_name  name_ratio
0              abe burrows' almanac       abe burrows almanac         97.0
1           action in the afternoon   action in the afternoon        100.0
18               adventure showcase            beat the clock        100.0
20   adventures of wild bill hickok               the big top         90.0
60                        air power         father knows best        100.0
87        alfred hitchcock presents            hotel de paree        100.0
255                      alkali ike         the liberace show        100.0
256             all around the town                the lineup        100.0
257      appointment with adventure          magnavox theater         95.0
311        armstrong circle theatre              valiant lady        100.0


In [205]:
df_right.to_csv("testing.csv")

In [155]:
#DOES NOT WORK
#Columns to match on from df_left

df_left.dropna().values
df_right.dropna().values

left_on = ["Program Title", "First Air Year"]

# Columns to match on from df_right
right_on = ["title", "production_year"]

fuzzymatcher.link_table(df_left, df_right, left_on, right_on)

ZeroDivisionError: float division by zero

In [158]:
#DOES NOT WORK
fuzzymatcher.fuzzy_left_join(df_left, df_right, left_on = "Program Title", right_on = "title")

ZeroDivisionError: float division by zero